In [1]:
import numpy as np

In [49]:
def CalculateDiffusionIncrements(CorrMatrix, dt, N):
    '''
    Given the sqare root of the correlation matrix and the size of time step, 
    calculate the values of dW_t or dB_t for all diffusion brownian motions in our model. 
    
    PARAMS:
    
    CorrMatrix: value of stochastic correlation matrix observation at time t.
    Size of the matrix is 2N+1 x 2N + 1. 
    
    dt: number, the size of time step of the simulation. 
    
    N:  the number of particles in our system. 
    '''
    iidStandartNormalVector = np.random.randn(2*N + 1)
    Increment = np.linalg.cholesky(CorrMatrix) @ iidStandartNormalVector * np.sqrt(dt)
    
    return Increment

In [59]:
class coef_function1d(object):
    '''
    1-dimensional coeffitient function. It depends on only 1 argument.
    Contains its value, first and second derivatives in point x. 
    '''
    
    def __init__(self, value, FirstDerivative, SecondDerivative):
        self.value = value
        self.dx    = FirstDerivative
        self.dxdx  = SecondDerivative
    
    def ReturnValue(self, x):
        return self.value(x)
    
    def dx(self, x):
        return self.FirstDerivative(x)
    
    def dxdx(self, x):
        return self.SecondDerivative(x)
    
    
class coef_function2d(object):
    '''
    2-dimensional coefficient function. Usually takes
    time-moment and process value as arguments. Both are real numbers.
    '''
    
    def __init__(self, value, Dt, Dx, DxDx):
        self.value = value
        self.Dt    = Dt
        self.Dx    = Dx
        self.DxDx  = Dxdx
    
    def value(self, t, x):
        return value(t, x)

    def dt(self, t, x):
        return self.Dt(t, x)
    
    def dx(self, t, x):
        return self.Dx(t, x)
    
    def dxdx(self, t, x):
        return self.DxDx(t, x)
    
class coef_function3d(object):
    '''
    3-dimensional coefficient function. Logic remains the same. 
    '''
    
    def __init__(self, value, Dt, Dx, Dy, DxDy, DxDx, DyDy):
        self.value = value
        self.Dt    = Dt
        self.Dx    = Dx
        self.Dy    = Dy
        self.DxDy  = Dxdy
        self.DxDx  = Dxdx
        self.DyDy  = Dydy
        
    def value(self, t, x, y):
        return self.value(t, x, y)
    
    def dt(self, t, x, y):
        return self.Dt(t, x, y)
    
    def dx(self, t, x, y):
        return self.Dx(t, x, y)
    
    def dy(self, t, x, y):
        return self.Dy(t, x, y)
    
    def dxdy(self, t, x, y):
        return self.DxDy(t, x, y)
    
    def dxdx(self, t, x, y):
        return self.DxDx(t, x, y)
    
    def dydy(self, t, x, y):
        return self.DyDy(t, x, y)   
    
    
class mean_reversion(object):
    '''
    Class for correlation mean-reverted drift function.
    It depends on N+1 arguments: nu1, ... , nuN, rho. 
    '''
    def __init__(self, value, gradient, hessian):
        self.value    = value
        self.gradient = gradient
        self.hessian  = hessian
        
    def dx(self, args, coordinate):
        return self.gradient[coordinate](args)
    
    def dxdx(self, args, i, j):
        return self.gessian[i][j](args)

In [ ]:
class ParticleProcess(object):
    '''
    This process describes the behaviour of the asset price. 
    
    The process can be described by an SDE:
    dS = drift_coef(t, S)dt + diffusion_coef(t, S, Nu)dWt, where Nu is the volatility process.
    
    NOT IMPLEMENTED YET
    
    PARAMS:
    S0:             the initial value of our diffusion. A number or a vector of numbers.
    drift_coef:     2d function, depends on time and process value. 
    diffusion_coef: 3d function, depends on time, process value and volatility value. 
    '''
    
    def __init__(self, S0, drift_coef, diffusion_coef):
        self.S0             = S0
        self.drift_coef     = drift_coef
        self.diffusion_coef = diffusion_coef
        
    def drift_coef_value(t, x):
        return drift_coef(t, x)
    
    def diffusion_coef_value(t, x, y):
        return diffusion_coef(t, x, y)
    

class VolatilityProcess(object):
    '''
    This process describes the behaviour of the asset price. 
    
    The process can be described by an SDE:
    dNu = drift_coef(t, Nu)dt + diffusion_coef(t, Nu)dBt, where Nu is the volatility process.
    
    NOT IMPLEMENTED YET
    
    PARAMS:
    Nu0:            the initial value of our diffusion. A number or a vector of numbers.
    drift_coef:     2d function, depends on time and volatility value. 
    diffusion_coef: 2d function, depends on time, and volatility value. 
    '''
    
    def __init__(self, Nu0, drift_coef, diffusion_coef):
        self.Nu0            = Nu0
        self.drift_coef     = drift_coef
        self.diffusion_coef = diffusion_coef
        
    def drift_coef_value(t, x):
        return drift_coef(t, x)
    
    def diffusion_coef_value(t, x):
        return diffusion_coef(t, x)
    

class CorrelationProcess(object):
    '''
    This process describes the behaviour of the asset price. 
    
    The process can be described by an SDE:
    dNu = drift_coef(t, Nu)dt + diffusion_coef(t, Nu)dBt, where Nu is the volatility process.
    
    NOT IMPLEMENTED YET
    
    PARAMS:
    rho0:           the initial value of our process. A number or a vector of numbers.
    mean_reversion: Psy(Nu1, ... , NuN) - rho_t. 
    diffusion_coef: 1d function, depends only on correlation value. 
    '''
    def __init__(self, rho0, alpha, mean_reversion, diffustion_coef):
        self.rho0           = rho0
        self.mean_reversion = mean_reversion
        self.diffusion_coef = diffusion_coef
        
    def drift_coef_value(args):
        '''
        PARAMS:
        args = (Nu1, ... , NuN, rho) -- volatilities and corr value at the moment. 
        '''
        return alpha * mean_reversion
    
    def diffusion_coef_value(rho):
        return diffusion_coef(rho)

In [57]:
def Jackson(x):
    return x**2 + x + 1

def dxJackson(x):
    return 2*x + 1

def dxdxJackson(x):
    return 2

f = function1d(Jackson, dxJackson, dxdxJackson)